In [ ]:
# Importing the required libraries, custom libraries and setting the python path
from load_env_and_set_pythonpath import print_pythonpath
import os
import inv_utils as iu
import helita_io_lp as lp
import time
import numpy as np
from skimage.transform import resize
import ambig_utils as au
from sunpy.coordinates import sun
from astropy.coordinates import SkyCoord
import astropy.units as u
from sunpy.coordinates import frames
import numpy as np
print_pythonpath()

In [ ]:
# Path of the ambig executable obtained from the compiling the ME0-2.1.1 Fortran code
ambig_executable_path = '/mn/stornext/u3/avijeetp/codes/ambiguity_resolution/ME0-2.1.1/'
# Path of the ambig_par file which is the input file for the ambig executable
ambig_par = '/mn/stornext/u3/avijeetp/codes/pyMilne/ITA/ambig_par'
# Load the fcube files obtained from the inversion
fblos = 'Blos_6173_2020-08-07_T083019_2020-08-07_T083019_t_12-12.fcube'
fbhor = 'Bhor_6173_2020-08-07_T083019_2020-08-07_T083019_t_12-12.fcube'
fbazi = 'Bazi_6173_2020-08-07_T083019_2020-08-07_T083019_t_12-12.fcube'

In [ ]:
# Load the configuration from the JSON file
input_config = iu.load_yaml_config('input_config.yaml')
# Check the input configuration
config = iu.check_input_config(input_config, pprint=False, confirm=False)

In [ ]:
# Load the configuration from the JSON file
crop = config['crop']
xorg = config['xorg']
xsize = config['xsize'] 
yorg = config['yorg']
ysize = config['ysize']
rescale = config['rescale']
is_north_up = config['is_north_up']
fov_angle = config['fov_angle']
fits_info = config['fits_info']
verbose = config['verbose']
save_dir = config['save_dir']
pix = config['scale']

In [ ]:
# Create a unique prefix for the output files based on the current time
id = int(time.time())
prefix = '_'.join(fblos.split('_')[1:-2]) + '_' + str(id) + '_'
print(f'suffix: {prefix}')

In [ ]:
# Obtain the time index from the file name
time_index = int(fblos.split('_')[-1].split('-')[0])
print(f'time index: {time_index}')

In [ ]:
# Get the range of latitudes and longitudes from the fits file info
tt = time_index
x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]
xcen = (x1 + x2) / 2
ycen = (y1 + y2) / 2
tobs = fits_info['all_start_times'][tt]

In [ ]:
# Load the data from the saved files
blos = lp.getdata(save_dir + fblos) # nx, ny, nt
bhor = lp.getdata(save_dir + fbhor)
bazi = lp.getdata(save_dir + fbazi)
# drop the redundant time axis
blos = blos.squeeze().T # ny, nx
bhor = bhor.squeeze().T
bazi = bazi.squeeze().T
# Plot the data
iu.plot_image(blos, figsize=(6, 6), title='Blos')

In [ ]:
# Crop the data if needed
if crop:
    blos = blos[yorg: yorg + ysize, xorg: xorg + xsize]
    bhor = bhor[yorg: yorg + ysize, xorg: xorg + xsize]
    bazi = bazi[yorg: yorg + ysize, xorg: xorg + xsize]
    if verbose:
        iu.plot_image(blos, figsize=(6, 6), title='Blos after cropping')

In [ ]:

if rescale:
    ysize, xsize = blos.shape 
    ysize = ysize // rescale
    xsize = xsize // rescale
    if verbose:
        print(f'Shape before rescale: {blos.shape}')
    # Resample the array to the new size
    blos = resize(blos, (ysize, xsize), anti_aliasing=True)
    bhor = resize(bhor, (ysize, xsize), anti_aliasing=True)
    bazi = resize(bazi, (ysize, xsize), anti_aliasing=True)
    if verbose:
        print(f'Shape after rescale: {blos.shape}')
        iu.plot_image(blos, figsize=(6, 6), title='Blos after rescaling')


In [ ]:

# Rotate the data to make it north-up
if not is_north_up:
    blos = iu.make_north_up(blos, fov_angle)
    bhor = iu.make_north_up(bhor, fov_angle)
    bazi = iu.make_north_up(bazi, fov_angle)
    ysize, xsize = blos.shape 
    if verbose:
        print(f'Shape after rotation: {blos.shape}')
    if ysize % 2 != 0 or xsize % 2 != 0:
        print('Resampling the array to ensure that the shape is a multiple of 2')
        # Resample the array to ensure that the shape is a multiple of 2
        ysize = ysize // 2 * 2
        xsize = xsize // 2 * 2
        blos = resize(blos, (ysize, xsize), anti_aliasing=True)
        bhor = resize(bhor, (ysize, xsize), anti_aliasing=True)
        bazi = resize(bazi, (ysize, xsize), anti_aliasing=True)
        if verbose:
            iu.plot_image(blos, figsize=(6, 6), title='Blos after rotation')

In [ ]:
# Calculate the P, B0 and angular radius of the Sun, and convert the coordinates of center to Heliographic Stonyhurst longitude and latitude
pbr = sun.P(tobs).value, sun.B0(tobs).value, sun.angular_radius(tobs).value
coord = SkyCoord(xcen*u.arcsec, ycen*u.arcsec, obstime=tobs,
                 frame=frames.Helioprojective, observer='earth')
coord_hs = coord.transform_to(frames.HeliographicStonyhurst)
lonlat = coord_hs.lon.value, coord_hs.lat.value

In [ ]:
# copy the ambig_par file to the save_dir with suffix appended
ambig_par_copy = save_dir  + prefix + 'par'
os.system(f'cp {ambig_par} {ambig_par_copy}')
if verbose:
    print(f'Copied {ambig_par} to {ambig_par_copy}')

In [ ]:
au.write_ambig_input('.', pix, pbr, lonlat, blos, bhor, bazi)
# copy the ambig_input file to the save_dir with suffix appended
ambig_input_copy = save_dir + prefix + 'ambig_input.dat'
os.system(f'cp {save_dir}ambig_input.dat {ambig_input_copy}')
if verbose:
    print(f'Copied {save_dir}ambig_input.dat to {ambig_input_copy}')

In [ ]:
# Run the ambig executable with the ambig_par file as input
command = ambig_executable_path + 'ambig ' + ambig_par
au.run_command(command)

In [ ]:
# Read the output files from the ambig executable
azimuth_dat = 'azimuth.dat'
# copy the azimuth.dat file to the save_dir with prefix appended to save_dir
azimuth_dat_copy = save_dir + prefix + 'azimuth.dat'
os.system(f'cp {azimuth_dat} {azimuth_dat_copy}')
if verbose:
    print(f'azimuth.dat copied to {azimuth_dat_copy}')

In [ ]:
phi = au.read_ambig_dat_file(azimuth_dat, (ysize, xsize))
if verbose:
    iu.plot_image(phi)
    print(f'phi shape: {phi.shape}')
    print(f'phi min: {phi.min()}')
    print(f'phi max: {phi.max()}')

In [ ]:
# Calculate the magnetic field components from the azimuthal angle
bx = bhor * np.cos(phi)
by = bhor * np.sin(phi)
bz = blos

In [ ]:
iu.plot_images([bx, by, bz, phi], title=['Bx', 'By','Bz', 'Phi'], fontsize=10, figsize=(8, 10), cmap=['gray_r', 'gray_r', 'gray_r', 'twilight_shifted'], grid_shape=(2, 2), fig_title=f'ID: {id}', save_fig=True, figname=save_dir + prefix + 'Bx_By_Bz_Phi.png')